## 0. Install and Import Dependencies

In [1]:
!pip install -q opencv-python mediapipe 


[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv.imshow("Mediapipe Feed", frame)

    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

## 1. Make Detections

In [14]:
cap = cv.VideoCapture(0)
# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv.imshow("Mediapipe Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

In [13]:
mp_drawing.DrawingSpec?

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
File:           ~/workspace/learning/ml/ml-learning/deep-learning/mediapipe/.venv/lib/python3.10/site-packages/mediapipe/python/solutions/drawing_utils.py
Type:           type
Subclasses:     

## 2. Determine Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [16]:
cap = cv.VideoCapture(0)
# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv.imshow("Mediapipe Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

[x: 0.42782417
y: 0.73361915
z: -0.6299564
visibility: 0.99976367
, x: 0.45467082
y: 0.6698297
z: -0.6130508
visibility: 0.9994331
, x: 0.47078714
y: 0.6683949
z: -0.61292124
visibility: 0.9995535
, x: 0.48489442
y: 0.66763914
z: -0.6130623
visibility: 0.9993917
, x: 0.41940284
y: 0.6716927
z: -0.59609544
visibility: 0.99946207
, x: 0.4109911
y: 0.67128277
z: -0.595306
visibility: 0.9995623
, x: 0.4038907
y: 0.6708785
z: -0.59557927
visibility: 0.9994635
, x: 0.51763135
y: 0.6898676
z: -0.37359542
visibility: 0.9994598
, x: 0.40937918
y: 0.6850686
z: -0.28793564
visibility: 0.99962056
, x: 0.45857817
y: 0.78944856
z: -0.5322531
visibility: 0.99972004
, x: 0.41607636
y: 0.7900857
z: -0.5075863
visibility: 0.9997435
, x: 0.63207334
y: 0.94608325
z: -0.1616419
visibility: 0.99689484
, x: 0.3353445
y: 0.91349596
z: -0.14547823
visibility: 0.9994116
, x: 0.67921203
y: 1.310273
z: -0.051199775
visibility: 0.13750648
, x: 0.22951937
y: 1.2120622
z: -0.09650765
visibility: 0.39275673
, x: 0.70

In [18]:
len(landmarks)

33

In [21]:
for landmark in mp_pose.PoseLandmark:
    print(landmark)

In [22]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.6052599
y: 0.95072657
z: -0.3049681
visibility: 0.92971903

In [30]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.6518736
y: 1.2987576
z: -0.33760422
visibility: 0.051167056

In [31]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.55531013
y: 1.4552648
z: -0.41185302
visibility: 0.01065574

## 3.Calculate Angles

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle
    
    return angle

In [39]:
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [37]:
shoulder, elbow, wrist

([0.605259895324707, 0.9507265686988831],
 [0.6518735885620117, 1.298757553100586],
 [0.5553101301193237, 1.4552648067474365])

In [36]:
calculate_angle(shoulder, elbow, wrist)

140.69727851421197

In [40]:
calculate_angle(hip, shoulder, elbow)

12.49435906476688

In [7]:
cap = cv.VideoCapture(0)
# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            print(angle)

            # Visualize angle
            cv.putText(image, str(angle), 
                       tuple(np.multiply(elbow, [1289, 720]).astype(int)),
                             cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv.LINE_AA
                       )

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv.imshow("Mediapipe Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world


In [46]:
tuple(np.multiply(elbow, [720, 1280]).astype(int))

(486, 1250)

## 4. Curl Counter

In [3]:
cap = cv.VideoCapture(0)

# Curl Counter variable
counter = 0
stage = None

# setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            # hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv.putText(image, str(angle), 
                       tuple(np.multiply(elbow, [1289, 720]).astype(int)),
                             cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv.LINE_AA
                       )

            # curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)

        except:
            pass

        # Render curl counter
        # Setup status box
        cv.rectangle(image, (0, 0), (255, 73), (245, 117, 16), -1)

        # Rep Data
        cv.putText(image, "REPS", 
            (15, 12),
            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA
        )
        cv.putText(image, str(counter), 
            (10, 60),
            cv.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv.LINE_AA
        )


        # Stage Data
        cv.putText(image, "Stage", 
            (95, 12),
            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv.LINE_AA
        )
        cv.putText(image, stage, 
            (90, 60),
            cv.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv.LINE_AA
        )
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv.imshow("Mediapipe Feed", image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
